Product registration consists of 5 steps, each presented by its own screen page. Data about each user and its conversion for each step and full funnel is presented in the attached file. Product manager wanted to try a revolutionary approach by redesigning the 3rd screen and expect to improve conversion at this step by 5pp.
It is requested from your side to prepare a calculation to answer the question about the existence of statistically significant difference between A and B group. Taking into consideration the main metric what AB affects is conversion at the 3rd step and secondary metric is full funnel pass rate.
Results should be prepared as a python script with output shown or a jupyter notebook. Statistical analysis should include calculation of test statistics, p-values and its interpretation.

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats

In [ ]:
file_path = '/content/conversions.csv'

In [ ]:
df_start = pd.read_csv(file_path, index_col=False)

In [ ]:
df = df_start.copy()

In [ ]:
df.head()

,user_id,group,page_visited
0,1,A,Splash
1,1,A,Mobile Phone
2,1,A,Passport
3,1,A,Online Banking
4,2,A,Splash


In [ ]:
df.isnull().sum()

user_id         0
group           0
page_visited    0
dtype: int64

In [ ]:
pivot_table = df_start.pivot_table(index='page_visited', columns='group', values='user_id', aggfunc='nunique')
pivot_table.sort_values(by=['A','B'], ascending=False)

group,A,B
page_visited,,
Splash,8998,1002
Mobile Phone,8531,960
Passport,7719,866
Online Banking,6532,770
Confirmation,5265,613


Для расчета CR третьего шага необходимо рассчитать отношение уникальных пользователей, перешедших на 4 экран(Online Banking), к числу уникальных пользователей, увидевших фичу на 3 экране (Passport)

In [ ]:
data_a_passport = df[(df['group'] == 'A') & (df['page_visited'] == 'Passport')]['user_id'].unique()
data_a_online_banking = df[(df['group'] == 'A') & (df['page_visited'] == 'Online Banking')]['user_id'].unique()
data_a = pd.Series([1 if user in data_a_online_banking else 0 for user in data_a_passport])

In [ ]:
data_b_passport = df[(df['group'] == 'B') & (df['page_visited'] == 'Passport')]['user_id'].unique()
data_b_online_banking = df[(df['group'] == 'B') & (df['page_visited'] == 'Online Banking')]['user_id'].unique()
data_b = pd.Series([1 if user in data_b_online_banking else 0 for user in data_b_passport])

In [ ]:
cr_a = np.mean(data_a)
cr_b = np.mean(data_b)

In [ ]:
print(f'Конверсия для группы A на третьем шаге: {cr_a:.2f}')
print(f'Конверсия для группы B на третьем шаге: {cr_b:.2f}')

Конверсия для группы A на третьем шаге: 0.85
Конверсия для группы B на третьем шаге: 0.89


In [ ]:
actual_increase = cr_b - cr_a

print(f'Фактическое увеличение CR на третьем шаге: {actual_increase:.2f}')

Фактическое увеличение CR на третьем шаге: 0.04


Применим t-test.
Почему не применяем непараметрический тест Манна Уитни можно почитать здесь: https://habr.com/ru/companies/avito/articles/709596/

In [ ]:
t_stat, p_value = stats.ttest_ind(data_a, data_b)

In [ ]:
print(f'T-test: t-statistic = {t_stat:.2f}, p-value = {p_value:.4f}')

T-test: t-statistic = -3.36, p-value = 0.0008


In [ ]:
alpha = 0.05
if p_value < alpha:
    print('Между контрольной и тестовой группами существует статистически значимое различие')
else:
    print('Между контрольной и тестовой группами статистически значимое различие не обнаружено')

Между контрольной и тестовой группами существует статистически значимое различие


Расчет прокси-метрики CR воронки

In [ ]:
data_a_splash = df_start[(df_start['group'] == 'A') & (df_start['page_visited'] == 'Splash')]['user_id'].unique()
data_a_confirm = df_start[(df_start['group'] == 'A') & (df_start['page_visited'] == 'Confirmation')]['user_id'].unique()

In [ ]:
data_a_funnel = pd.Series([1 if user in data_a_confirm else 0 for user in data_a_splash])

In [ ]:
data_b_splash = df_start[(df_start['group'] == 'B') & (df_start['page_visited'] == 'Splash')]['user_id'].unique()
data_b_confirm = df_start[(df_start['group'] == 'B') & (df_start['page_visited'] == 'Confirmation')]['user_id'].unique()

In [ ]:
data_b_funnel = pd.Series([1 if user in data_b_confirm else 0 for user in data_b_splash])

In [ ]:
cr_a_funnel = np.mean(data_a_funnel)
cr_b_funnel = np.mean(data_b_funnel)

In [ ]:
print(f'Конверсия воронки для контрольной группы: {cr_a_funnel:.2f}')
print(f'Конверсия воронки для тестовой группы: {cr_b_funnel:.2f}')

Конверсия воронки для контрольной группы: 0.59
Конверсия воронки для тестовой группы: 0.61


In [ ]:
actual_increase_funnel = cr_b_funnel - cr_a_funnel

print(f'Фактическое увеличение CR воронки {actual_increase_funnel:.2f}')

Фактическое увеличение CR воронки 0.03


Фактическое увеличение составило 4%, что является статистически значимым результатом (p-value < 0.05). Конверсия воронки также выросла на 3% в тестовой группе.
Несмотря на то, что редизайн показал положительные изменения, статистически значимое увеличение конверсии составило 4%, что не дотягивает до MDE = 5%, установленного менеджером.

Рекомендации:
Увеличить размер выборки и повторить эксперимент, чтобы добиться более точных результатов